# Lakeflow Declarative Pipeline Exercise - Requirements

## Overview

You are tasked with building a **Lakeflow Declarative Pipeline** to process e-commerce data from raw JSON files. The data contains various **data quality issues** that you must identify and resolve as you build your pipeline through the Bronze, Silver, and Gold layers.

### Learning Objectives

By completing this exercise, you will:
- Design and implement a Bronze layer to ingest raw data
- Design and implement a Silver layer to clean and transform data
- Design and implement a Gold layer to create analytical tables for dashboards
- Handle data quality issues including duplicates, missing values, invalid formats, and data type inconsistencies
- Create a Lakeflow Declarative Pipeline using SQL or PySpark (Delta Live Tables)

---

## Prerequisites

1. **REQUIRED**: Run `00_LDP_Exercise_Setup_Environment.ipynb` first to set up the environment and generate sample data
2. Familiarity with SQL and data transformation concepts
3. Understanding of medallion architecture (Bronze, Silver, Gold)

---

## Data Sources

Your raw data is located in Unity Catalog volumes at:
```
/Volumes/ldp_exercise/exercise_schema/raw/
```

The following data sources are available:

1. **Products** (`products/00.json`)
   - Product catalog information
   - Fields: `product_id`, `name`, `category`, `price`, `sku`, `stock_quantity`, `created_at`

2. **Customers** (`customers/00.json`)
   - Customer master data
   - Fields: `customer_id`, `name`, `email`, `address`, `city`, `state`, `zip_code`, `phone`, `created_at`

3. **Orders** (`orders/00.json`)
   - Order transactions
   - Fields: `order_id`, `customer_id`, `order_date`, `status`, `total_amount`

4. **Payments** (`payments/00.json`)
   - Payment transactions
   - Fields: `payment_id`, `order_id`, `amount`, `payment_method`, `payment_status`, `payment_date`

---

## ⚠️ Data Quality Issues

**IMPORTANT**: The sample data contains intentional data quality issues. You must identify and handle these issues in your pipeline:

- **Duplicates**: Some records appear multiple times
- **Missing Values**: NULL values in critical fields
- **Invalid Formats**: Incorrect date formats, invalid email addresses, negative prices
- **Data Type Issues**: Numbers stored as strings, dates as strings in wrong format
- **Inconsistent Data**: Mixed case values, inconsistent state codes
- **Orphaned Records**: References to non-existent entities (e.g., orders referencing non-existent customers)
- **Invalid Values**: Negative amounts, zero prices, invalid status codes

**Your task**: Identify these issues and implement appropriate data quality checks and transformations in your pipeline.

---

## Exercise Requirements

### Bronze Layer (Your Design)

**No specific instructions provided** - You must design this layer yourself.

Consider:
- How to ingest raw JSON files from the volume
- What metadata to capture (file names, processing timestamps, etc.)
- Whether to use streaming tables or materialized views
- How to handle schema evolution

### Silver Layer (Your Design)

**No specific instructions provided** - You must design this layer yourself.

Consider:
- How to clean and validate data
- How to handle duplicates
- How to standardize data formats (dates, strings, numbers)
- How to handle missing values
- How to validate referential integrity (e.g., orders must reference valid customers)
- How to filter out invalid records or flag them appropriately
- Data type conversions and standardization

### Gold Layer (Specific Requirements)

You **MUST** create the following analytical tables/views for dashboard consumption:

#### 1. **Sales Summary Dashboard Table**

Table Name: `sales_summary_dashboard`

This table should provide daily sales metrics:

**Required Columns:**
- `sale_date` (DATE) - The date of the sale
- `total_revenue` (DECIMAL) - Total revenue for the day (sum of order amounts)
- `total_orders` (BIGINT) - Total number of orders for the day
- `avg_order_value` (DECIMAL) - Average order value for the day
- `total_customers` (BIGINT) - Number of unique customers who placed orders
- `completed_orders` (BIGINT) - Number of orders with status 'delivered'
- `cancelled_orders` (BIGINT) - Number of orders with status 'cancelled'

**Business Rules:**
- Only include orders with valid, positive `total_amount`
- Only include orders with status 'delivered', 'shipped', 'processing', or 'cancelled'
- Exclude orders with NULL or invalid dates
- Calculate metrics based on the order date

---

#### 2. **Customer Analytics Dashboard Table**

Table Name: `customer_analytics_dashboard`

This table should provide customer-level analytics:

**Required Columns:**
- `customer_id` (BIGINT) - Customer identifier
- `customer_name` (STRING) - Customer full name
- `customer_state` (STRING) - Customer state (standardized to uppercase)
- `total_orders` (BIGINT) - Total number of orders placed by customer
- `total_spent` (DECIMAL) - Total amount spent by customer across all orders
- `avg_order_value` (DECIMAL) - Average order value for the customer
- `first_order_date` (DATE) - Date of customer's first order
- `last_order_date` (DATE) - Date of customer's most recent order
- `customer_lifetime_value` (DECIMAL) - Same as total_spent (for consistency)

**Business Rules:**
- Only include customers with valid email addresses (must contain '@')
- Only include customers with complete address information (address, city, state, zip_code all not NULL)
- Only count orders with valid, positive amounts
- Only count orders with status 'delivered' or 'shipped'
- Handle duplicate customers (keep the most recent record based on created_at)
- Standardize state codes to uppercase

---

#### 3. **Product Performance Dashboard Table**

Table Name: `product_performance_dashboard`

This table should provide product-level performance metrics:

**Required Columns:**
- `product_id` (BIGINT) - Product identifier
- `product_name` (STRING) - Product name
- `category` (STRING) - Product category
- `price` (DECIMAL) - Product price
- `stock_quantity` (BIGINT) - Current stock quantity
- `total_revenue` (DECIMAL) - Total revenue generated by this product (from orders)
- `total_units_sold` (BIGINT) - Total units sold (if available from order items, otherwise NULL)
- `is_in_stock` (BOOLEAN) - Whether product is in stock (stock_quantity > 0)
- `is_active` (BOOLEAN) - Whether product is active (price > 0 and valid SKU)

**Business Rules:**
- Only include products with valid, positive prices
- Only include products with non-empty SKU
- Set `stock_quantity` to 0 if it's negative
- Calculate revenue from orders (you may need to infer this from order amounts if order items are not available)
- If order items are not available, set `total_units_sold` to NULL

---

#### 4. **Payment Analytics Dashboard Table**

Table Name: `payment_analytics_dashboard`

This table should provide payment transaction analytics:

**Required Columns:**
- `payment_date` (DATE) - Date of the payment
- `payment_method` (STRING) - Payment method used
- `total_transactions` (BIGINT) - Total number of transactions
- `total_amount` (DECIMAL) - Total amount processed
- `successful_transactions` (BIGINT) - Number of transactions with status 'completed'
- `failed_transactions` (BIGINT) - Number of transactions with status 'failed'
- `refunded_transactions` (BIGINT) - Number of transactions with status 'refunded'
- `success_rate` (DECIMAL) - Percentage of successful transactions (successful / total * 100)

**Business Rules:**
- Only include payments with valid, positive amounts
- Only include payments linked to valid orders (order_id exists in orders table)
- Only include payments with valid payment methods (credit_card, debit_card, paypal, bank_transfer, cash)
- Exclude payments with NULL or invalid dates
- Calculate success rate as: (successful_transactions / total_transactions) * 100

---

## Technical Requirements

1. **Pipeline Structure**:
   - Create your pipeline file(s) in the `solution/` folder
   - You can use either:
     - **SQL**: Lakeflow Spark Declarative Pipeline SQL syntax (`.sql` file)
     - **PySpark**: Delta Live Tables (DLT) with PySpark (`.py` file)
   - Follow medallion architecture: Bronze → Silver → Gold

2. **Table Naming Convention**:
   - Bronze tables: `{entity}_bronze` (e.g., `orders_bronze`, `customers_bronze`)
   - Silver tables: `{entity}_silver` (e.g., `orders_silver`, `customers_silver`)
   - Gold tables: Use the exact names specified above for dashboard tables

3. **Catalog and Schema**:
   - All tables must be created in: `ldp_exercise.exercise_schema`

4. **Data Quality**:
   - Implement appropriate data quality checks
   - Handle or filter invalid records
   - Document your data quality decisions in comments

5. **Performance**:
   - Use appropriate table types (STREAMING TABLE for Bronze/Silver, MATERIALIZED VIEW for Gold)
   - Consider partitioning if needed
   - Use TBLPROPERTIES to mark quality layers

---

## Deliverables

1. **Lakeflow Declarative Pipeline file(s)** in the `solution/` folder
   - Your pipeline should process all four data sources (products, customers, orders, payments)
   - Implement Bronze, Silver, and Gold layers
   - Create all four required Gold layer dashboard tables
   - You can submit either:
     - SQL file(s) using Lakeflow Spark Declarative Pipeline syntax (`.sql`)
     - PySpark file(s) using Delta Live Tables (`.py`)
     - Or both if you want to demonstrate proficiency in both approaches

2. **Documentation** (optional but recommended):
   - Comments in your code explaining data quality decisions
   - Brief explanation of your Bronze and Silver layer design choices

---

## Evaluation Criteria

Your solution will be evaluated on:

1. **Correctness**: All four Gold layer tables must match the specified schema and business rules
2. **Data Quality**: Proper handling of data quality issues (duplicates, nulls, invalid formats, etc.)
3. **Architecture**: Proper implementation of medallion architecture (Bronze → Silver → Gold)
4. **Code Quality**: Clean, readable code (SQL or PySpark) with appropriate comments
5. **Completeness**: All data sources processed and integrated appropriately

---

## Getting Started

1. Review the data by exploring the raw JSON files in the volume
2. Identify the data quality issues present in each data source
3. Design your Bronze layer to ingest raw data
4. Design your Silver layer to clean and transform data
5. Implement the Gold layer according to the specifications above
6. Test your pipeline and verify the Gold layer tables

---

## Tips

- Start by exploring the raw data to understand its structure and issues
- Use `STREAM read_files()` for Bronze layer ingestion
- Consider using `DISTINCT` or window functions to handle duplicates
- Use `TRY_CAST` or `CAST` for data type conversions with error handling
- Use `FILTER` or `WHERE` clauses to exclude invalid records
- Test incrementally: build Bronze first, then Silver, then Gold
- Use the Databricks SQL editor or notebook cells to test your queries before creating the pipeline

---

**Good luck!** 🚀